# Modelo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### Célula 1: Montando o Google Drive
Esta célula importa a biblioteca `drive` do `google.colab` e a usa para montar seu Google Drive no ambiente do Colab. Isso permite que o notebook acesse arquivos armazenados em seu Drive.

### Célula 2: Copiando o arquivo ZIP
Esta célula usa um comando shell (`!cp`) para copiar um arquivo ZIP chamado `nfl-big-data-bowl-2026-prediction.zip` de uma pasta específica no seu Google Drive (`/content/drive/MyDrive/'Colab Notebooks'/`) para o diretório raiz do ambiente Colab (`/content/`). Isso é necessário para descompactar o arquivo no ambiente local.

### Célula 3: Descompactando o arquivo
Esta célula usa um comando shell (`!unzip`) para descompactar o arquivo `nfl-big-data-bowl-2026-prediction.zip` que foi copiado para `/content/`. A saída mostra todos os arquivos e diretórios que foram extraídos, incluindo dados de treinamento (`train/input_*.csv`, `train/output_*.csv`) e arquivos de avaliação (`test.csv`, `kaggle_evaluation/`).

### Célula 4: Listando arquivos de treinamento
Esta célula usa um comando shell (`!ls`) para listar o conteúdo do diretório `/content/train`. Isso serve para verificar se os arquivos de entrada e saída do conjunto de treinamento foram descompactados corretamente e estão acessíveis.

### Célula 5: Pré-processamento, Engenharia de Features e Treinamento do Modelo
Esta é a célula principal que realiza a maior parte do trabalho, desde o carregamento dos dados até o treinamento do modelo. Vamos dividi-la em seções:

#### 5.1. Importações e Carregamento de Dados
```python
import pandas as pd
import numpy as np
import glob

from sklearn.model_selection import GroupKFold
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor


inputs = []
outputs = []

for f in glob.glob("/content/train/input_*.csv"):
    inputs.append(pd.read_csv(f))

for f in glob.glob("/content/train/output_*.csv"):
    outputs.append(pd.read_csv(f))

input_df = pd.concat(inputs, ignore_index=True)
output_df = pd.concat(outputs, ignore_index=True)
```
*   **Importações**: Importa bibliotecas essenciais como `pandas` para manipulação de dados, `numpy` para operações numéricas, `glob` para encontrar arquivos, e classes do `sklearn` e `xgboost` para modelagem.
*   **Carregamento e Concatenação**: O código itera sobre todos os arquivos `input_*.csv` e `output_*.csv` no diretório `/content/train`, lê cada um deles para um DataFrame e os concatena em `input_df` e `output_df` respectivamente. Esses DataFrames contêm os dados de entrada (posições dos jogadores, velocidade, etc.) e os dados de saída (posições alvo x e y da bola, número de frames futuros) para cada jogada.

#### 5.2. Engenharia de Features de Lag
```python
input_df = input_df.sort_values(
    ["game_id","play_id","nfl_id","frame_id"]
)

for lag in [1,2,3,5]:
    input_df[f"x_lag{lag}"] = input_df.groupby(
        ["game_id","play_id","nfl_id"]
    )["x"].shift(lag)

    input_df[f"y_lag{lag}"] = input_df.groupby(
        ["game_id","play_id","nfl_id"]
    )["y"].shift(lag)

    input_df[f"s_lag{lag}"] = input_df.groupby(
        ["game_id","play_id","nfl_id"]
    )["s"].shift(lag)
```
*   **Ordenação**: O `input_df` é ordenado por `game_id`, `play_id`, `nfl_id` e `frame_id` para garantir que as operações de `shift` (lag) funcionem corretamente.
*   **Recursos de Lag**: São criadas novas features (`x_lag`, `y_lag`, `s_lag`) que representam os valores de `x`, `y` e `s` (velocidade) de frames anteriores. Isso é feito agrupando por `game_id`, `play_id` e `nfl_id` para garantir que o lag seja aplicado dentro do contexto de cada jogador em cada jogada, e usando `shift()` com diferentes valores de `lag` (1, 2, 3 e 5 frames).

#### 5.3. Preparação dos Dados para Treinamento
```python
key = ["game_id","play_id","nfl_id"]

last_input = (
    input_df
    .loc[
        input_df.groupby(key)["frame_id"].idxmax()
    ]
)

df = output_df.merge(
    last_input,
    on=key,
    suffixes=("_target","_input")
)
df["k"] = df["frame_id_target"]
```
*   **`last_input`**: Seleciona a última entrada de `frame_id` para cada combinação única de `game_id`, `play_id` e `nfl_id` no `input_df`.
*   **Junção (Merge)**: O `output_df` é mesclado com `last_input` usando `game_id`, `play_id` e `nfl_id` como chaves. Os sufixos `_target` e `_input` são adicionados para diferenciar colunas com nomes semelhantes que vêm de `output_df` e `last_input`, respectivamente. `df['k']` é criado a partir de `frame_id_target` (que representa o número de frames à frente que estamos prevendo).

#### 5.4. Mais Engenharia de Features
```python
df["dt"] = df["num_frames_output"]
df["dx"] = df["x_target"] - df["x_input"]
df["dy"] = df["y_target"] - df["y_input"]
df["dir_rad"] = np.deg2rad(df["dir"])
df["vx"] = df["s"] * np.cos(df["dir_rad"])
df["vy"] = df["s"] * np.sin(df["dir_rad"])
df["k2"] = df["k"]**2
df["vx_k"] = df["vx"] * df["k"]
df["vy_k"] = df["vy"] * df["k"]
df["a_k2"] = df["a"] * df["k"]**2
df["o_rad"] = np.deg2rad(df["o"])
df["ox"] = np.cos(df["o_rad"])
df["oy"] = np.sin(df["o_rad"])

# Vetor até a bola
df["dist_ball_x"] = df["ball_land_x"] - df["x_input"]
df["dist_ball_y"] = df["ball_land_y"] - df["y_input"]

# Distância
df["dist_ball"] = np.sqrt(
    df["dist_ball_x"]**2 + df["dist_ball_y"]**2
)

# Ângulo até a bola
df["angle_ball"] = np.arctan2(
    df["dist_ball_y"],
    df["dist_ball_x"]
)

# Alinhamento com velocidade
df["dot_ball_vel"] = (
    df["dist_ball_x"] * df["vx"] +
    df["dist_ball_y"] * df["vy"]
)
```
*   **Diferenças e Velocidades**: Calcula a diferença nas posições (`dx`, `dy`), converte direção (`dir`) e orientação (`o`) para radianos e componentes X/Y (`vx`, `vy`, `ox`, `oy`).
*   **Features de Tempo**: Cria `k2`, `vx_k`, `vy_k`, `a_k2` que envolvem o número de frames futuros `k`.
*   **Relação com a Bola**: Calcula a distância e o ângulo do jogador em relação ao ponto de aterrissagem da bola (`ball_land_x`, `ball_land_y`), além do produto escalar do vetor para a bola com o vetor velocidade do jogador (`dot_ball_vel`).

#### 5.5. Tratamento de Variáveis Categóricas e Altura
```python
cat_cols = [
    "player_position",
    "player_role",
    "player_side",
    "play_direction"
]
def height_to_inches(h):
    try:
        feet, inch = h.split("-")
        return int(feet)*12 + int(inch)
    except:
        return np.nan

df["player_height_in"] = df["player_height"].apply(height_to_inches)

df["player_height_in"] = df["player_height_in"].fillna(
    df["player_height_in"].median()
)
for c in cat_cols:
    df[c] = df[c].astype("category").cat.codes
```
*   **`height_to_inches`**: Uma função para converter a altura do jogador (formatada como "pés-polegadas") para polegadas. Lida com possíveis erros retornando `NaN`.
*   **Preenchimento de Nulos**: Valores `NaN` na altura são preenchidos com a mediana da coluna.
*   **Codificação Categórica**: As colunas categóricas (`player_position`, `player_role`, `player_side`, `play_direction`) são convertidas para códigos numéricos usando `astype("category").cat.codes`.

#### 5.6. Definição de Features e Variáveis Alvo
```python
features = [
    "x_input","y_input",
    "vx","vy","a","k","k2","vx_k","vy_k","a_k2",
    "o_rad","ox","oy",
    "player_weight",
     "player_height_in",
    "player_position",
    "player_role",
    "player_side",
    "play_direction",
    "x_lag1","x_lag2","x_lag3","x_lag5",
    "y_lag1","y_lag2","y_lag3","y_lag5",
    "s_lag1","s_lag2","s_lag3","s_lag5",
      "ball_land_x",
    "ball_land_y",
    "dist_ball_x",
    "dist_ball_y",
    "dist_ball",
    "angle_ball",
    "dot_ball_vel"

]

df["dx_n"] = df["dx"] / df["k"]
df["dy_n"] = df["dy"] / df["k"]

y = df[["dx_n","dy_n"]]
groups = (
    df["game_id"].astype(str)
    + "_"
    + df["play_id"].astype(str)
)
```
*   **`features`**: Uma lista de todas as colunas que serão usadas como features para o modelo, incluindo as originais e as engenheiradas.
*   **Variáveis Alvo Normalizadas**: `dx_n` e `dy_n` são calculadas dividindo `dx` e `dy` por `k` (o número de frames futuros). Isso normaliza o deslocamento pela duração da previsão, tornando o problema de regressão mais estável.
*   **`y`**: O DataFrame das variáveis alvo normalizadas (`dx_n`, `dy_n`).
*   **`groups`**: Uma string que combina `game_id` e `play_id`. Isso é usado para `GroupKFold` na validação cruzada para garantir que os dados de uma mesma jogada (`game_id`, `play_id`) não sejam divididos entre os conjuntos de treino e validação, evitando vazamento de dados.

#### 5.7. Ajuste para Direção da Jogada
```python
left = df["play_direction"] == "left"
df.loc[left, "ball_land_x"] = 120 - df.loc[left, "ball_land_x"]


for col in ["x_input","x_lag1","x_lag2","x_lag3","x_lag5"]:
    df.loc[left, col] = 120 - df.loc[left, col]

df.loc[left, "vx"] *= -1


X = df[features]
```
*   **Inversão para Lado Esquerdo**: Para jogadas que se movem para a esquerda, as coordenadas X (incluindo `ball_land_x`, `x_input`, e `x_lag` features) são invertidas em relação ao comprimento do campo (120 jardas). A velocidade `vx` também é invertida. Isso é uma técnica comum para padronizar a direção do campo, tratando jogadas para a esquerda e para a direita de forma simétrica e simplificando o aprendizado do modelo.
*   **`X`**: O DataFrame final das features para o modelo.

#### 5.8. Métrica de Avaliação
```python
def metric(y_true, y_pred):
    return np.sqrt(
        np.mean(
            (y_true[:,0]-y_pred[:,0])**2 +
            (y_true[:,1]-y_pred[:,1])**2
        ) / 2
    )
```
*   Define uma função `metric` que calcula a raiz do erro quadrático médio (RMSE) para duas dimensões (x e y), dividido por 2. Esta métrica avalia a distância euclidiana média entre as previsões e os valores reais.

#### 5.9. Validação Cruzada e Treinamento do Modelo
```python
gkf = GroupKFold(n_splits=5)

scores = []

for fold, (tr, va) in enumerate(gkf.split(X, y, groups)):

    X_tr, X_va = X.iloc[tr], X.iloc[va]

    y_tr_x = y.iloc[tr, 0]   # dx
    y_tr_y = y.iloc[tr, 1]   # dy

    y_va_x = y.iloc[va, 0]
    y_va_y = y.iloc[va, 1]

    model_x = XGBRegressor(
     n_estimators=2000,
     max_depth=6,
     learning_rate=0.05,
     subsample=0.8,
     colsample_bytree=0.8,
     tree_method="hist",
     random_state=42,
     eval_metric="rmse",
     early_stopping_rounds=50
    )


    model_y = XGBRegressor(
      n_estimators=2000,
      max_depth=6,
      learning_rate=0.05,
      subsample=0.8,
      colsample_bytree=0.8,
      tree_method="hist",
      random_state=42,
      eval_metric="rmse",
      early_stopping_rounds=50
)



    # Treina separado
    model_x.fit(
    X_tr, y_tr_x,
    eval_set=[(X_va, y_va_x)],

    verbose=False
)
    model_y.fit(X_tr, y_tr_y,
                eval_set=[(X_va, y_va_y)],
                verbose=False)

    # Prediz separado
    pred_x = model_x.predict(X_va)
    pred_y = model_y.predict(X_va)

    # Junta
    pred = np.column_stack([pred_x, pred_y])

    k_val = df.iloc[va]["k"].values.reshape(-1,1)

    pred_real = pred * k_val
    y_real = df.iloc[va][["dx","dy"]].values

    sc = metric(y_real, pred_real)
    scores.append(sc)

    print(f"Fold {fold}: {sc:.4f}")


print("CV mean:", np.mean(scores))
```
*   **GroupKFold**: Inicializa `GroupKFold` com 5 divisões. Isso garante que os dados de uma mesma jogada fiquem sempre no mesmo conjunto (treino ou validação) em cada fold, evitando vazamento de informação.
*   **Loop de Validação Cruzada**: O código itera por cada fold:
    *   Divide os dados em conjuntos de treino (`X_tr`, `y_tr`) e validação (`X_va`, `y_va`).
    *   Separa as variáveis alvo `dx` e `dy` (normalizadas) para treinamento individual.
    *   **Modelos XGBoost**: São criados dois modelos `XGBRegressor` idênticos, um para prever `dx_n` (`model_x`) e outro para `dy_n` (`model_y`).
        *   Os parâmetros incluem `n_estimators` (número de árvores), `max_depth`, `learning_rate`, `subsample`, `colsample_bytree`, `tree_method="hist"` (para desempenho), `random_state` e `early_stopping_rounds` (para parar o treinamento se o desempenho na validação não melhorar).
    *   **Treinamento**: Cada modelo é treinado em seu respectivo alvo (`y_tr_x` ou `y_tr_y`), usando o conjunto de validação (`eval_set`) para monitorar o desempenho e aplicar o `early_stopping`.
    *   **Previsão**: Após o treinamento, os modelos fazem previsões nos dados de validação (`pred_x`, `pred_y`).
    *   **Reversão da Normalização**: As previsões normalizadas (`pred`) são multiplicadas de volta por `k_val` (o número de frames futuros) para obter os deslocamentos reais previstos (`pred_real`), que podem ser comparados com os deslocamentos reais `y_real` (`dx`, `dy`).
    *   **Cálculo da Métrica**: A métrica de avaliação (`metric`) é calculada para o fold atual e adicionada à lista `scores`.
    *   **Impressão dos Resultados**: Imprime a métrica para cada fold e, no final, a média das métricas de todos os folds (`CV mean`), fornecendo uma estimativa robusta do desempenho do modelo.

In [ ]:
!cp /content/drive/MyDrive/'Colab Notebooks'/nfl-big-data-bowl-2026-prediction.zip /content/



In [ ]:
!unzip  /content/nfl-big-data-bowl-2026-prediction.zip

Archive:  /content/nfl-big-data-bowl-2026-prediction.zip
  inflating: kaggle_evaluation/__init__.py  
  inflating: kaggle_evaluation/core/__init__.py  
  inflating: kaggle_evaluation/core/base_gateway.py  
  inflating: kaggle_evaluation/core/generated/__init__.py  
  inflating: kaggle_evaluation/core/generated/kaggle_evaluation_pb2.py  
  inflating: kaggle_evaluation/core/generated/kaggle_evaluation_pb2_grpc.py  
  inflating: kaggle_evaluation/core/kaggle_evaluation.proto  
  inflating: kaggle_evaluation/core/relay.py  
  inflating: kaggle_evaluation/core/templates.py  
  inflating: kaggle_evaluation/nfl_gateway.py  
  inflating: kaggle_evaluation/nfl_inference_server.py  
  inflating: test.csv                
  inflating: test_input.csv          
  inflating: train/input_2023_w01.csv  
  inflating: train/input_2023_w02.csv  
  inflating: train/input_2023_w03.csv  
  inflating: train/input_2023_w04.csv  
  inflating: train/input_2023_w05.csv  
  inflating: train/input_2023_w06.csv  
  

In [ ]:
!ls /content/train

input_2023_w01.csv  input_2023_w13.csv	 output_2023_w07.csv
input_2023_w02.csv  input_2023_w14.csv	 output_2023_w08.csv
input_2023_w03.csv  input_2023_w15.csv	 output_2023_w09.csv
input_2023_w04.csv  input_2023_w16.csv	 output_2023_w10.csv
input_2023_w05.csv  input_2023_w17.csv	 output_2023_w11.csv
input_2023_w06.csv  input_2023_w18.csv	 output_2023_w12.csv
input_2023_w07.csv  output_2023_w01.csv  output_2023_w13.csv
input_2023_w08.csv  output_2023_w02.csv  output_2023_w14.csv
input_2023_w09.csv  output_2023_w03.csv  output_2023_w15.csv
input_2023_w10.csv  output_2023_w04.csv  output_2023_w16.csv
input_2023_w11.csv  output_2023_w05.csv  output_2023_w17.csv
input_2023_w12.csv  output_2023_w06.csv  output_2023_w18.csv


In [ ]:
import pandas as pd
import numpy as np
import glob

from sklearn.model_selection import GroupKFold
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor


inputs = []
outputs = []

for f in glob.glob("/content/train/input_*.csv"):
    inputs.append(pd.read_csv(f))

for f in glob.glob("/content/train/output_*.csv"):
    outputs.append(pd.read_csv(f))

input_df = pd.concat(inputs, ignore_index=True)
output_df = pd.concat(outputs, ignore_index=True)

input_df = input_df.sort_values(
    ["game_id","play_id","nfl_id","frame_id"]
)

for lag in [1,2,3,5]:
    input_df[f"x_lag{lag}"] = input_df.groupby(
        ["game_id","play_id","nfl_id"]
    )["x"].shift(lag)

    input_df[f"y_lag{lag}"] = input_df.groupby(
        ["game_id","play_id","nfl_id"]
    )["y"].shift(lag)

    input_df[f"s_lag{lag}"] = input_df.groupby(
        ["game_id","play_id","nfl_id"]
    )["s"].shift(lag)


key = ["game_id","play_id","nfl_id"]

last_input = (
    input_df
    .loc[
        input_df.groupby(key)["frame_id"].idxmax()
    ]
)


df = output_df.merge(
    last_input,
    on=key,
    suffixes=("_target","_input")
)
df["k"] = df["frame_id_target"]

df[["x_input","x_target"]].head()

df["dt"] = df["num_frames_output"]
df["dx"] = df["x_target"] - df["x_input"]
df["dy"] = df["y_target"] - df["y_input"]
df["dir_rad"] = np.deg2rad(df["dir"])
df["vx"] = df["s"] * np.cos(df["dir_rad"])
df["vy"] = df["s"] * np.sin(df["dir_rad"])
df["k2"] = df["k"]**2
df["vx_k"] = df["vx"] * df["k"]
df["vy_k"] = df["vy"] * df["k"]
df["a_k2"] = df["a"] * df["k"]**2
df["o_rad"] = np.deg2rad(df["o"])
df["ox"] = np.cos(df["o_rad"])
df["oy"] = np.sin(df["o_rad"])

# Vetor até a bola
df["dist_ball_x"] = df["ball_land_x"] - df["x_input"]
df["dist_ball_y"] = df["ball_land_y"] - df["y_input"]

# Distância
df["dist_ball"] = np.sqrt(
    df["dist_ball_x"]**2 + df["dist_ball_y"]**2
)

# Ângulo até a bola
df["angle_ball"] = np.arctan2(
    df["dist_ball_y"],
    df["dist_ball_x"]
)

# Alinhamento com velocidade
df["dot_ball_vel"] = (
    df["dist_ball_x"] * df["vx"] +
    df["dist_ball_y"] * df["vy"]
)



cat_cols = [
    "player_position",
    "player_role",
    "player_side",
    "play_direction"
]
def height_to_inches(h):
    try:
        feet, inch = h.split("-")
        return int(feet)*12 + int(inch)
    except:
        return np.nan

df["player_height_in"] = df["player_height"].apply(height_to_inches)

df["player_height_in"] = df["player_height_in"].fillna(
    df["player_height_in"].median()
)
for c in cat_cols:
    df[c] = df[c].astype("category").cat.codes
features = [
    "x_input","y_input",
    "vx","vy","a","k","k2","vx_k","vy_k","a_k2",
    "o_rad","ox","oy",
    "player_weight",
     "player_height_in",
    "player_position",
    "player_role",
    "player_side",
    "play_direction",
    "x_lag1","x_lag2","x_lag3","x_lag5",
    "y_lag1","y_lag2","y_lag3","y_lag5",
    "s_lag1","s_lag2","s_lag3","s_lag5",
      "ball_land_x",
    "ball_land_y",
    "dist_ball_x",
    "dist_ball_y",
    "dist_ball",
    "angle_ball",
    "dot_ball_vel"

]


df["dx_n"] = df["dx"] / df["k"]
df["dy_n"] = df["dy"] / df["k"]

y = df[["dx_n","dy_n"]]
groups = (
    df["game_id"].astype(str)
    + "_"
    + df["play_id"].astype(str)
)


left = df["play_direction"] == "left"
df.loc[left, "ball_land_x"] = 120 - df.loc[left, "ball_land_x"]


for col in ["x_input","x_lag1","x_lag2","x_lag3","x_lag5"]:
    df.loc[left, col] = 120 - df.loc[left, col]

df.loc[left, "vx"] *= -1


X = df[features]
def metric(y_true, y_pred):
    return np.sqrt(
        np.mean(
            (y_true[:,0]-y_pred[:,0])**2 +
            (y_true[:,1]-y_pred[:,1])**2
        ) / 2
    )

gkf = GroupKFold(n_splits=5)

scores = []

for fold, (tr, va) in enumerate(gkf.split(X, y, groups)):

    X_tr, X_va = X.iloc[tr], X.iloc[va]

    y_tr_x = y.iloc[tr, 0]   # dx
    y_tr_y = y.iloc[tr, 1]   # dy

    y_va_x = y.iloc[va, 0]
    y_va_y = y.iloc[va, 1]

    model_x = XGBRegressor(
     n_estimators=2000,
     max_depth=6,
     learning_rate=0.05,
     subsample=0.8,
     colsample_bytree=0.8,
     tree_method="hist",
     random_state=42,
     eval_metric="rmse",
     early_stopping_rounds=50
    )


    model_y = XGBRegressor(
      n_estimators=2000,
      max_depth=6,
      learning_rate=0.05,
      subsample=0.8,
      colsample_bytree=0.8,
      tree_method="hist",
      random_state=42,
      eval_metric="rmse",
      early_stopping_rounds=50
)



    # Treina separado
    model_x.fit(
    X_tr, y_tr_x,
    eval_set=[(X_va, y_va_x)],

    verbose=False
)
    model_y.fit(X_tr, y_tr_y,
                eval_set=[(X_va, y_va_y)],
                verbose=False)

    # Prediz separado
    pred_x = model_x.predict(X_va)
    pred_y = model_y.predict(X_va)

    # Junta
    pred = np.column_stack([pred_x, pred_y])

    k_val = df.iloc[va]["k"].values.reshape(-1,1)

    pred_real = pred * k_val
    y_real = df.iloc[va][["dx","dy"]].values

    sc = metric(y_real, pred_real)
    scores.append(sc)

    print(f"Fold {fold}: {sc:.4f}")


print("CV mean:", np.mean(scores))




Fold 0: 1.4565
Fold 1: 0.9392
Fold 2: 0.8857
Fold 3: 0.8618
Fold 4: 0.8810
CV mean: 1.0048338484670425
